# Reproducing the benchmark model in Keras

In this notebook I reacreate the models from [here](https://github.com/emilwallner/Coloring-greyscale-images/blob/master/Beta-version/beta_version.ipynb) in order to see if I can replicate it and use it as a benchmark.

## Setup

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install gdown

# create directory for storing data
!mkdir -p images

# download zip file with training set
!gdown https://drive.google.com/uc?id=1rTf4DJI2PoM-hgSSSzvpOQcQhui3qh-y && unzip -qq train.zip -d images
!rm train.zip

# download zip with valid set
!gdown https://drive.google.com/uc?id=1yB-KN--FjiWZrA6XwxaSvtXhtcbk80dQ && unzip -qq valid.zip -d images
!rm valid.zip

# change dir to the one with data 
!cd /content

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, InputLayer, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import h5py

from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave, imread, imshow
from skimage.transform import resize

import numpy as np
import os
import random

In [ ]:
IMG_SIZE = 256
N_EPOCHS = 100
BATCH_SIZE = 32

## Preparing the Data

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   horizontal_flip = True)

train_set = train_datagen.flow_from_directory('images/train_small/',
                                              target_size = (IMG_SIZE, IMG_SIZE),
                                              batch_size = BATCH_SIZE)

valid_datagen = ImageDataGenerator(rescale = 1./255)

valid_set = valid_datagen.flow_from_directory('images/valid_small/',
                                              target_size = (IMG_SIZE, IMG_SIZE),
                                              batch_size = BATCH_SIZE)

def gen_ab_images(train_set):
    for batch in train_set:
        lab_batch = rgb2lab(batch[0])
        X_batch = lab_batch[:,:,:,0] / 100
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)



## Model v0 (alpha)

In [ ]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [ ]:
model.summary()

## Model v1 (beta)

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [ ]:
model.summary()

## Training the model

In [ ]:
tensorboard = TensorBoard(log_dir="output/first_run")
model.fit(x=gen_ab_images(train_set), 
          callbacks=[tensorboard], 
          epochs=N_EPOCHS, 
          validation_data=gen_ab_images(valid_set), 
          steps_per_epoch=len(train_set),
          validation_steps=len(valid_set))

In [ ]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

## Evaluation

In [ ]:
batch = next(gen_ab_images(valid_set))
Xtest, Ytest = batch[0], batch[1]

In [ ]:
print(model.evaluate(Xtest, Ytest, batch_size=BATCH_SIZE))

In [ ]:
# Test model
output = model.predict(Xtest)
output = output * 128

# Output colorizations
i = random.randrange(len(output))
cur = np.zeros((IMG_SIZE, IMG_SIZE, 3))
cur[:,:,0] = Xtest[i][:,:,0] * 100
cur[:,:,1:] = output[i]
cur = lab2rgb(cur)
imshow(cur)